In [45]:
import requests
import pandas as pd
headers = {
    'user-agent': 'my-app'
}


def main(url):
    with requests.Session() as req:
        req.headers.update(headers)
        allin = []
        for t in ['AMZN', 'GS']:
            params = {
                't': t
            }
            r = req.get(url, params=params)
            df = pd.read_html(r.content, attrs={'id': 'news-table'})[0]
            allin.append(df)
        df = pd.concat(allin, ignore_index=True)
        print(df[0])
        #df.index = list('date','title')

main('https://finviz.com/quote.ashx')

0      Jan-16-23 01:27PM
1                12:45PM
2                11:36AM
3                11:29AM
4                10:34AM
             ...        
201              10:33AM
202              09:00AM
203              07:45AM
204              07:40AM
205    Nov-28-22 01:09PM
Name: 0, Length: 206, dtype: object


In [65]:
df = pd.read_csv('data.csv', usecols=[0,1], names=['date_time', 'title'])

In [83]:
df

,date_time,title
0,0,1
1,Jan-16-23 01:27PM,2 FAANG Stocks to Buy in 2023 and 1 to Avoid: ...
2,12:45PM,Davos: PwC survey finds 'bleak CEO outlook' fo...
3,11:36AM,2 Stocks to Benefit From Falling Inflation Gur...
4,11:29AM,Big Tech Stocks Could Rebound Big Time. Heres ...
...,...,...
202,10:33AM,Goldman Sachs sees less pain but also no gain ...
203,09:00AM,"The Goldman Sachs Group, Inc.'s (NYSE:GS) Low ..."
204,07:45AM,Stack Capital Invests $8 Million USD Into Locu...
205,07:40AM,Abu Dhabi's ADNOC working with Goldman Sachs o...


In [87]:
# drop first row
data=df.drop(index=0)
data

,date_time,title
1,Jan-16-23 01:27PM,2 FAANG Stocks to Buy in 2023 and 1 to Avoid: ...
2,12:45PM,Davos: PwC survey finds 'bleak CEO outlook' fo...
3,11:36AM,2 Stocks to Benefit From Falling Inflation Gur...
4,11:29AM,Big Tech Stocks Could Rebound Big Time. Heres ...
5,10:34AM,Amazon Has a Partner for Enterprise Blockchain...
...,...,...
202,10:33AM,Goldman Sachs sees less pain but also no gain ...
203,09:00AM,"The Goldman Sachs Group, Inc.'s (NYSE:GS) Low ..."
204,07:45AM,Stack Capital Invests $8 Million USD Into Locu...
205,07:40AM,Abu Dhabi's ADNOC working with Goldman Sachs o...


In [86]:
data.iloc[2][0]  

'11:36AM'

In [70]:
type(df.iloc[1][1])


str

In [91]:
parsed_data = []
        
for i, row in df.iterrows():
    title = df['title'].iloc[i]
    date_data = df.iloc[i][0].split(' ')
    if len(date_data) == 1:
        time = date_data[0] #timestamp = row.td.text
    else:
        date = date_data[0]
        time = date_data[1]
        
    parsed_data.append([date, time, title])

clean_df = pd.DataFrame(parsed_data)
clean_df

,0,1,2
0,Nov-28-22,0,1
1,Jan-16-23,01:27PM,2 FAANG Stocks to Buy in 2023 and 1 to Avoid: ...
2,Jan-16-23,12:45PM,Davos: PwC survey finds 'bleak CEO outlook' fo...
3,Jan-16-23,11:36AM,2 Stocks to Benefit From Falling Inflation Gur...
4,Jan-16-23,11:29AM,Big Tech Stocks Could Rebound Big Time. Heres ...
...,...,...,...
202,Nov-29-22,10:33AM,Goldman Sachs sees less pain but also no gain ...
203,Nov-29-22,09:00AM,"The Goldman Sachs Group, Inc.'s (NYSE:GS) Low ..."
204,Nov-29-22,07:45AM,Stack Capital Invests $8 Million USD Into Locu...
205,Nov-29-22,07:40AM,Abu Dhabi's ADNOC working with Goldman Sachs o...
